> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 3.14 - ORM

## Exercícios

#### Q1. Conhecendo os dados
Baixe o seguinte csv onde iremos trabalhar. Ele contém informações sobre salários de profissionais de dados de uma empresa hipotética entre 2009 e 2016
* https://github.com/camilalaranjeira/python-intermediario/blob/main/salaries.csv

Suas colunas, descritas na [página do Kaggle que contém o dataset](https://www.kaggle.com/datasets/krishujeniya/salary-prediction-of-data-professions?resource=download), são:
* FIRST NAME: Primeiro nome do profissional de dados (String)
* LAST NAME: Sobrenome do profissional de dados (String)
* SEX: Gênero do profissional de dados (String: 'F' para Feminino, 'M' para Masculino)
* DOJ (Date of Joining): A data em que o profissional de dados ingressou na empresa (Data no formato MM/DD/AAAA)
* CURRENT DATE: A data atual ou a data de referência dos dados (Data no formato MM/DD/AAAA)
* DESIGNATION: O cargo ou designação do profissional de dados (String: ex., Analista, Analista Sênior, Gerente)
* AGE: Idade do profissional de dados (Integer)
* SALARY: Salário anual do profissional de dados (Float)
* UNIT: Unidade de negócios ou departamento em que o profissional de dados trabalha (String: ex., TI, Finanças, Marketing)
* LEAVES USED: Número de licenças utilizadas pelo profissional de dados (Integer)
* LEAVES REMAINING: Número de licenças restantes para o profissional de dados (Integer)
* RATINGS: Avaliações de desempenho do profissional de dados (Float)
* PAST EXP: Experiência de trabalho anterior em anos antes de ingressar na empresa atual (Float)

Na célula a seguir, **carregue os dados do CSV e dê uma olhada neles antes de seguir**.

In [1]:
### Escreva sua resposta aqui
import pandas as pd

# URL do CSV
url = "https://github.com/camilalaranjeira/python-intermediario/raw/main/salaries.csv"

# Carregando os dados
df = pd.read_csv(url)

# Mostrando as primeiras linhas
print(df.head())

# Informações gerais sobre o dataframe
print(df.info())

# Estatísticas descritivas das colunas numéricas
print(df.describe())


  FIRST NAME   LAST NAME SEX         DOJ CURRENT DATE DESIGNATION   AGE  \
0     TOMASA       ARMEN   F   5-18-2014   01-07-2016     Analyst  21.0   
1      ANNIE         NaN   F         NaN   01-07-2016   Associate   NaN   
2      OLIVE        ANCY   F   7-28-2014   01-07-2016     Analyst  21.0   
3     CHERRY     AQUILAR   F  04-03-2013   01-07-2016     Analyst  22.0   
4       LEON  ABOULAHOUD   M  11-20-2014   01-07-2016     Analyst   NaN   

   SALARY        UNIT  LEAVES USED  LEAVES REMAINING  RATINGS  PAST EXP  
0   44570     Finance         24.0               6.0      2.0         0  
1   89207         Web          NaN              13.0      NaN         7  
2   40955     Finance         23.0               7.0      3.0         0  
3   45550          IT         22.0               8.0      3.0         0  
4   43161  Operations         27.0               3.0      NaN         3  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2639 entries, 0 to 2638
Data columns (total 13 columns)

#### Q2. Modelando os dados
Você deve **criar um ORM com SQLAlchemy capaz de comportar os dados dessa base**.

* Crie um campo de chave primária `ID`, que deve ser incrementado automaticamente
* Os campos SEX, DESIGNATION e UNIT devem ser definidos como classes `Enum` com os possíveis valores (consulte os valores únicos dessas colunas)
* Para os outros campos, consulte os tipos de dados informados na descrição acima

In [2]:
### Escreva sua resposta aqui
from sqlalchemy import create_engine, Column, Integer, Float, String, Date, Enum
from sqlalchemy.ext.declarative import declarative_base
import pandas as pd
import enum

# Base para os modelos
Base = declarative_base()

# --- Carregar dados para pegar valores únicos dos enums ---
url = "https://github.com/camilalaranjeira/python-intermediario/raw/main/salaries.csv"
df = pd.read_csv(url)

# Pegando valores únicos para os Enums
sex_values = df['SEX'].dropna().unique().tolist()
designation_values = df['DESIGNATION'].dropna().unique().tolist()
unit_values = df['UNIT'].dropna().unique().tolist()

# Criando enums Python
class SexEnum(enum.Enum):
    # cria dinamicamente
    pass

for v in sex_values:
    setattr(SexEnum, v, v)

class DesignationEnum(enum.Enum):
    pass

for v in designation_values:
    key = v.replace(" ", "_").replace("-", "_")  # nomes válidos de atributo
    setattr(DesignationEnum, key, v)

class UnitEnum(enum.Enum):
    pass

for v in unit_values:
    key = v.replace(" ", "_").replace("-", "_")
    setattr(UnitEnum, key, v)

# --- Modelo ORM ---
class Salary(Base):
    __tablename__ = 'salaries'

    id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String)
    last_name = Column(String)
    sex = Column(Enum(SexEnum))
    doj = Column(Date)  # Date of Joining
    current_date = Column(Date)
    designation = Column(Enum(DesignationEnum))
    age = Column(Integer)
    salary = Column(Float)
    unit = Column(Enum(UnitEnum))
    leaves_used = Column(Integer)
    leaves_remaining = Column(Integer)
    ratings = Column(Float)
    past_exp = Column(Float)

# --- Criando a engine (SQLite em memória, só como exemplo) ---
engine = create_engine('sqlite:///:memory:', echo=True)
Base.metadata.create_all(engine)


2025-12-04 16:58:43,512 INFO sqlalchemy.engine.Engine BEGIN (implicit)


/tmp/ipython-input-900434690.py:8: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-12-04 16:58:43,515 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("salaries")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("salaries")


2025-12-04 16:58:43,516 INFO sqlalchemy.engine.Engine [raw sql] ()


...

INFO:sqlalchemy.engine.Engine:
CREATE TABLE salaries (
	id INTEGER NOT NULL, 
	first_name VARCHAR, 
	last_name VARCHAR, 
	sex VARCHAR, 
	doj DATE, 
	"current_date" DATE, 
	designation VARCHAR, 
	age INTEGER, 
	salary FLOAT, 
	unit VARCHAR, 
	leaves_used INTEGER, 
	leaves_remaining INTEGER, 
	ratings FLOAT, 
	past_exp FLOAT, 
	PRIMARY KEY (id)
)




2025-12-04 16:58:43,525 INFO sqlalchemy.engine.Engine [no key 0.00222s] ()


INFO:sqlalchemy.engine.Engine:[no key 0.00222s] ()


2025-12-04 16:58:43,527 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


#### Q3. Estabelecendo uma conexão

Usando o método `create_engine` do SQLAlchemy, crie uma conexão com um novo banco de dados SQLite chamado `salarios`.

In [3]:
### Escreva sua resposta aqui
from sqlalchemy import create_engine

# Criando a engine SQLite
engine = create_engine('sqlite:///salarios.db', echo=True)

# Testando a conexão (opcional)
connection = engine.connect()
print("Conexão com o banco 'salarios.db' estabelecida com sucesso!")

# Fechando a conexão de teste
connection.close()


Conexão com o banco 'salarios.db' estabelecida com sucesso!


#### Q4. Criando as tabelas
Crie as tabelas da questão Q2 no banco `salarios`.

In [4]:
### Escreva sua resposta aqui
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, Float, String, Date, Enum
import enum
import pandas as pd

# --- Base para os modelos ---
Base = declarative_base()

# --- Carregar dados do CSV para pegar valores únicos dos enums ---
url = "https://github.com/camilalaranjeira/python-intermediario/raw/main/salaries.csv"
df = pd.read_csv(url)

sex_values = df['SEX'].dropna().unique().tolist()
designation_values = df['DESIGNATION'].dropna().unique().tolist()
unit_values = df['UNIT'].dropna().unique().tolist()

# --- Criando Enums ---
class SexEnum(enum.Enum):
    pass
for v in sex_values:
    setattr(SexEnum, v, v)

class DesignationEnum(enum.Enum):
    pass
for v in designation_values:
    key = v.replace(" ", "_").replace("-", "_")
    setattr(DesignationEnum, key, v)

class UnitEnum(enum.Enum):
    pass
for v in unit_values:
    key = v.replace(" ", "_").replace("-", "_")
    setattr(UnitEnum, key, v)

# --- Modelo ORM ---
class Salary(Base):
    __tablename__ = 'salaries'

    id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String)
    last_name = Column(String)
    sex = Column(Enum(SexEnum))
    doj = Column(Date)
    current_date = Column(Date)
    designation = Column(Enum(DesignationEnum))
    age = Column(Integer)
    salary = Column(Float)
    unit = Column(Enum(UnitEnum))
    leaves_used = Column(Integer)
    leaves_remaining = Column(Integer)
    ratings = Column(Float)
    past_exp = Column(Float)

# --- Criando a engine SQLite ---
from sqlalchemy import create_engine
engine = create_engine('sqlite:///salarios.db', echo=True)

# --- Criando as tabelas no banco ---
Base.metadata.create_all(engine)
print("Tabelas criadas com sucesso no banco 'salarios.db'!")


2025-12-04 16:59:56,180 INFO sqlalchemy.engine.Engine BEGIN (implicit)


/tmp/ipython-input-744438363.py:8: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-12-04 16:59:56,182 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("salaries")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("salaries")


2025-12-04 16:59:56,185 INFO sqlalchemy.engine.Engine [raw sql] ()


...

2025-12-04 16:59:56,196 INFO sqlalchemy.engine.Engine [no key 0.00300s] ()


INFO:sqlalchemy.engine.Engine:[no key 0.00300s] ()


2025-12-04 16:59:56,216 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


Tabelas criadas com sucesso no banco 'salarios.db'!


#### Q5. Populando

Usando o método `to_sql` da biblioteca Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)), popule o banco `salarios` com os dados do csv que você carregou na questão Q1.
* Lembre-se de definir o parâmetro `if_exists='append'` para que as tabelas não sejam dropadas e recriadas.

In [6]:
# Renomear colunas para o padrão usado no ORM
df_renamed = df.rename(columns={
    "FIRST NAME": "first_name",
    "LAST NAME": "last_name",
    "SEX": "sex",
    "DOJ": "doj",
    "CURRENT DATE": "current_date",
    "DESIGNATION": "designation",
    "AGE": "age",
    "SALARY": "salary",
    "UNIT": "unit",
    "LEAVES USED": "leaves_used",
    "LEAVES REMAINING": "leaves_remaining",
    "RATINGS": "ratings",
    "PAST EXP": "past_exp"
})

# Converter colunas Enum para string
df_renamed['sex'] = df_renamed['sex'].astype(str)
df_renamed['designation'] = df_renamed['designation'].astype(str)
df_renamed['unit'] = df_renamed['unit'].astype(str)

# Converter datas
df_renamed['doj'] = pd.to_datetime(df_renamed['doj'], format='%m/%d/%Y', errors='coerce')
df_renamed['current_date'] = pd.to_datetime(df_renamed['current_date'], format='%m/%d/%Y', errors='coerce')

# Inserir no banco
df_renamed.to_sql('salaries', con=engine, if_exists='append', index=False)
print("Dados inseridos com sucesso!")


2025-12-04 17:01:53,343 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-12-04 17:01:53,366 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("salaries")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("salaries")


2025-12-04 17:01:53,378 INFO sqlalchemy.engine.Engine [raw sql] ()


...

2025-12-04 17:01:53,690 INFO sqlalchemy.engine.Engine [generated in 0.23030s] [('TOMASA', 'ARMEN', 'F', None, None, 'Analyst', 21.0, 44570, 'Finance', 24.0, 6.0, 2.0, 0), ('ANNIE', None, 'F', None, None, 'Associate', None, 89207, 'Web', None, 13.0, None, 7), ('OLIVE', 'ANCY', 'F', None, None, 'Analyst', 21.0, 40955, 'Finance', 23.0, 7.0, 3.0, 0), ('CHERRY', 'AQUILAR', 'F', None, None, 'Analyst', 22.0, 45550, 'IT', 22.0, 8.0, 3.0, 0), ('LEON', 'ABOULAHOUD', 'M', None, None, 'Analyst', None, 43161, 'Operations', 27.0, 3.0, None, 3), ('VICTORIA', None, 'F', None, None, 'Analyst', 22.0, 48736, 'Marketing', 20.0, 10.0, 4.0, 0), ('ELLIOT', 'AGULAR', 'M', None, None, 'Analyst', 22.0, 40339, 'Marketing', 19.0, 11.0, 5.0, 0), ('JACQUES', 'AKMAL', 'M', None, None, 'Analyst', None, 40058, 'Marketing', 29.0, 1.0, 2.0, 2)  ... displaying 10 of 2639 total bound parameter sets ...  ('TERI', 'ANASTASIO', 'F', None, None, 'Analyst', 24.0, 45172, 'Web', 23.0, 7.0, 3.0, 1), ('GREGORY', 'ABARCA', 'M', Non

INFO:sqlalchemy.engine.Engine:[generated in 0.23030s] [('TOMASA', 'ARMEN', 'F', None, None, 'Analyst', 21.0, 44570, 'Finance', 24.0, 6.0, 2.0, 0), ('ANNIE', None, 'F', None, None, 'Associate', None, 89207, 'Web', None, 13.0, None, 7), ('OLIVE', 'ANCY', 'F', None, None, 'Analyst', 21.0, 40955, 'Finance', 23.0, 7.0, 3.0, 0), ('CHERRY', 'AQUILAR', 'F', None, None, 'Analyst', 22.0, 45550, 'IT', 22.0, 8.0, 3.0, 0), ('LEON', 'ABOULAHOUD', 'M', None, None, 'Analyst', None, 43161, 'Operations', 27.0, 3.0, None, 3), ('VICTORIA', None, 'F', None, None, 'Analyst', 22.0, 48736, 'Marketing', 20.0, 10.0, 4.0, 0), ('ELLIOT', 'AGULAR', 'M', None, None, 'Analyst', 22.0, 40339, 'Marketing', 19.0, 11.0, 5.0, 0), ('JACQUES', 'AKMAL', 'M', None, None, 'Analyst', None, 40058, 'Marketing', 29.0, 1.0, 2.0, 2)  ... displaying 10 of 2639 total bound parameter sets ...  ('TERI', 'ANASTASIO', 'F', None, None, 'Analyst', 24.0, 45172, 'Web', 23.0, 7.0, 3.0, 1), ('GREGORY', 'ABARCA', 'M', None, None, 'Analyst', 24.0

2025-12-04 17:01:53,806 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


Dados inseridos com sucesso!


#### Q6. Consultas SQL vs ORM

Agrupe os dados por DESIGNATION e selecione o mínimo, máximo e a média dos salários (SALARY) divididos por 12. Já que o atributo SALARY é anual, dividir por 12 nos mostrará os valores mensais.

Assumindo que a variável que armazena a sua conexão se chama `engine`, você deve realizar a query acima de três formas:
* Executando a query SQL através de uma instância de conexão retornada pelo método `engine.connect()`
* Executando a query SQL com o método `read_sql_query` do Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html)). Você usará mesma instância `engine.connect()` como um dos parâmetros.
* Executando uma query criada com o módulo `select` do SQLAlchemy. Sua execução deve ser feita através de um objeto `Session` do módulo `orm` do SQLAlchemy (`Session(engine)`).


In [7]:
### Execute aqui sua query SQL com SQLAlchemy
from sqlalchemy import text

with engine.connect() as conn:
    query = text("""
        SELECT
            designation,
            MIN(salary)/12 AS min_salary,
            MAX(salary)/12 AS max_salary,
            AVG(salary)/12 AS avg_salary
        FROM salaries
        GROUP BY designation
    """)
    result = conn.execute(query)
    for row in result:
        print(row)


2025-12-04 17:02:48,926 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-12-04 17:02:48,933 INFO sqlalchemy.engine.Engine 
        SELECT 
            designation,
            MIN(salary)/12 AS min_salary,
            MAX(salary)/12 AS max_salary,
            AVG(salary)/12 AS avg_salary
        FROM salaries
        GROUP BY designation
    


INFO:sqlalchemy.engine.Engine:
        SELECT 
            designation,
            MIN(salary)/12 AS min_salary,
            MAX(salary)/12 AS max_salary,
            AVG(salary)/12 AS avg_salary
        FROM salaries
        GROUP BY designation
    


2025-12-04 17:02:48,938 INFO sqlalchemy.engine.Engine [generated in 0.01142s] ()


INFO:sqlalchemy.engine.Engine:[generated in 0.01142s] ()


('Analyst', 3333.4166666666665, 4165.0, 3751.675987685993)
('Associate', 5846.166666666667, 8300.25, 7266.915094339623)
('Director', 17832.25, 32342.666666666668, 23914.265625)
('Manager', 8343.666666666666, 12407.5, 10522.716049382716)
('Senior Analyst', 4170.333333333333, 5830.5, 4991.778792134832)
('Senior Manager', 12614.416666666666, 16631.416666666668, 14888.689516129032)
2025-12-04 17:02:48,952 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


In [8]:
### Execute aqui sua query SQL com SQLAlchemy + Pandas
import pandas as pd

with engine.connect() as conn:
    df_salary = pd.read_sql_query("""
        SELECT
            designation,
            MIN(salary)/12 AS min_salary,
            MAX(salary)/12 AS max_salary,
            AVG(salary)/12 AS avg_salary
        FROM salaries
        GROUP BY designation
    """, conn)

print(df_salary)


2025-12-04 17:03:01,787 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-12-04 17:03:01,791 INFO sqlalchemy.engine.Engine 
        SELECT 
            designation,
            MIN(salary)/12 AS min_salary,
            MAX(salary)/12 AS max_salary,
            AVG(salary)/12 AS avg_salary
        FROM salaries
        GROUP BY designation
    


INFO:sqlalchemy.engine.Engine:
        SELECT 
            designation,
            MIN(salary)/12 AS min_salary,
            MAX(salary)/12 AS max_salary,
            AVG(salary)/12 AS avg_salary
        FROM salaries
        GROUP BY designation
    


2025-12-04 17:03:01,793 INFO sqlalchemy.engine.Engine [raw sql] ()


INFO:sqlalchemy.engine.Engine:[raw sql] ()


2025-12-04 17:03:01,801 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


      designation    min_salary    max_salary    avg_salary
0         Analyst   3333.416667   4165.000000   3751.675988
1       Associate   5846.166667   8300.250000   7266.915094
2        Director  17832.250000  32342.666667  23914.265625
3         Manager   8343.666667  12407.500000  10522.716049
4  Senior Analyst   4170.333333   5830.500000   4991.778792
5  Senior Manager  12614.416667  16631.416667  14888.689516


In [11]:
from sqlalchemy.orm import Session
from sqlalchemy import select, func

with Session(engine) as session:
    stmt = (
        select(
            Salary.designation,
            (func.min(Salary.salary)/12).label('min_salary'),
            (func.max(Salary.salary)/12).label('max_salary'),
            (func.avg(Salary.salary)/12).label('avg_salary')
        )
        .group_by(Salary.designation)
    )

    result = session.execute(stmt).all()

# Mostrar resultado
for row in result:
    print(row)


2025-12-04 17:04:54,772 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-12-04 17:04:54,781 INFO sqlalchemy.engine.Engine SELECT salaries.designation, min(salaries.salary) / (? + 0.0) AS min_salary, max(salaries.salary) / (? + 0.0) AS max_salary, avg(salaries.salary) / (? + 0.0) AS avg_salary 
FROM salaries GROUP BY salaries.designation


INFO:sqlalchemy.engine.Engine:SELECT salaries.designation, min(salaries.salary) / (? + 0.0) AS min_salary, max(salaries.salary) / (? + 0.0) AS max_salary, avg(salaries.salary) / (? + 0.0) AS avg_salary 
FROM salaries GROUP BY salaries.designation


2025-12-04 17:04:54,784 INFO sqlalchemy.engine.Engine [generated in 0.00286s] (12, 12, 12)


INFO:sqlalchemy.engine.Engine:[generated in 0.00286s] (12, 12, 12)


2025-12-04 17:04:54,791 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


('Analyst', 3333.4166666666665, 4165.0, 3751.675987685993)
('Associate', 5846.166666666667, 8300.25, 7266.915094339623)
('Director', 17832.25, 32342.666666666668, 23914.265625)
('Manager', 8343.666666666666, 12407.5, 10522.716049382716)
('Senior Analyst', 4170.333333333333, 5830.5, 4991.778792134832)
('Senior Manager', 12614.416666666666, 16631.416666666668, 14888.689516129032)
